In [1]:
import numpy as np
import pandas as pd

## Tidy data
[Hadley Wickham](https://en.wikipedia.org/wiki/Hadley_Wickham) wrote a [great article](https://www.jstatsoft.org/article/view/v059i10) in favor of "tidy data." Tidy `DataFrames` follow these rules:

1. Each variable is a column.
2. Each observation is a row.
3. Each type of observation has its own separate `DataFrame`.

This is less pretty to visualize as a table, but we rarely look at data in tables. Indeed, the representation of data which is convenient for visualization is different from that which is convenient for analysis. A tidy `DataFrame` is almost always **much** easier to work with than non-tidy formats.

You may raise some objections about tidy data. Here are a few, and my responses.

**Objection:** Looking at a table of tidy data is ugly. It is not intuitively organized. I would almost never display a tidy data table in a publication.

**Response:** Correct! Having tabular data in a format that is easy to read as a human studying a table is a very different thing than having it in a format that is easy to explore and work with using a computer. As my friend Daniel Chen put it, "There are data formats that are better for reporting and data formats that are better for analysis." We are using the tidy data frames for analysis, not reporting (though we will see in the coming lessons that having the data in a tidy format makes making plots much easier, and plots are a key medium for reporting.)


**Objection:** Isn't it better to sometimes have data arranged in other ways? Say in a matrix?

**Response:** This is certainly true for things like images, or raster-style data in general. It makes more sense to organize an image in a 2D matrix than to have it organized as a data frame with three columns (row in image, column in image, intensity of pixel), where each row corresponds to a single pixel. For an image, indexing it by row and column is always unambiguous, `my_image[i, j]` means the pixel at row `i` and column `j`.
For other data, though, the matrix layout suffers from the fact that there may be more than one way to construct a matrix. If you know a data frame is tidy, you already know its structure. You need only to ask what the columns are, and then you immediately know how to access data. In other formats, you might have to read and write extensive comments to understand the structure of the data. Of course, you can read and write comments, but it opens the door for the possibility of misinterpretation or mistakes.


**Objection:** But what about time series? Clearly, that can be in matrix format. One column is time, and then subsequent columns are observations made at that time.

**Response:** Yes, that is true. But then the matrix-style described could be considered tidy, since each row is a single observation (time point) that has many facets.


**Objection:** Isn't this an inefficient use of memory? There tend to be lots of repeated entries in tidy data frames.

**Response:** Yes, there are more efficient ways of storing and accessing data. But for data sets that are not "big data," this is seldom a real issue. The extra expense in memory, as well as the extra expense in access, is a small prices to pay for the simplicity and speed of the human user in accessing the data.


**Objection:** Once it's tidy, we pretty much have to use Boolean indexing to get what we want, and that can be slower than other methods of accessing data. What about performance?

**Response:** See the previous response. Speed of access really only because of a problem with big, high-throughput data sets. In those cases, there are often many things you need to be clever about.


**Conclusion:** I really think that tidying a data set allows for fluid exploration. We will focus on tidy data sets going forward. The techniques for bringing untidy data into tidy format use many of Pandas's functions, but are largely beyond the scope of this bootcamp. You will explore that a little bit in the exercises, but for more of the bootcamp, our data sets are already tidy.

## The data set
We will again use the data set from the [Beattie, et al. paper](http://rsos.royalsocietypublishing.org/content/3/10/160321) on facial matching under sleep deprivation. Let's load in the original data set and add the column on insomnia as we did in thelast lesson.

In [3]:
df = pd.read_csv('../data/gfmt_sleep.csv', na_values='*')
df['insomnia'] = df['sci'] <= 16

# Take a look
df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True


This data set is in tidy format. Each row represents a single test on a single participant. The aspects of that person's test are given in each column. We already saw the power of having the data in this format when we did Boolean indexing in the last lesson. Now, we will see how this format allows use to easily do an operation we do again and again with data sets, split-apply-combine.


## Split-apply-combine
Let's say we want to compute the median percent correct face matchings for subjects with insomnia and the median percent correct face matchings for those without. Ignoring for the second the mechanics of how we would do this with Python, let's think about it in English. What do we need to do?

1. **Split** the data set up according to the `'insomnia'` field, i.e., split it up so we have a separate data set for the two classes of subjects, those with insomnia and those without.
2. **Apply** a median function to the activity in these split data sets.
3. **Combine** the results of these averages on the split data set into a new, summary data set that contains the two classes (insomniac and not) and means for each.

We see that the strategy we want is a **split-apply-combine** strategy. This idea was put forward by Hadley Wickham in [this paper](https://www.jstatsoft.org/article/view/v040i01). It turns out that this is a strategy we want to use *very* often. Split the data in terms of some criterion. (We may want to split by genotype.) Apply some function to the split-up data. Combine the results into a new data frame.

Note that if the data are tidy, this procedure makes a lot of sense. Choose the column you want to use to split by. All rows with like entries in the splitting column are then grouped into a new data set. You can then apply any function you want into these new data sets. You can then combine the results into a new data frame.
Pandas's split-apply-combine operations are achieved using the `groupby()` method. You can think of `groupby()` as the splitting part. You can then apply functions to the resulting `DataFrameGroupBy` object. The [Pandas documentation on split-apply-combine](http://pandas.pydata.org/pandas-docs/stable/groupby.html) is excellent and worth reading through. It is extensive though, so don't let yourself get intimidated by it.

## Aggregation: Median percent correct
Let's go ahead and do our first split-apply-combine on this tidy data set. First, we will split the data set up by `insomnia` condition.

In [11]:
grouped = df.groupby('insomnia')

# Take a look
grouped

There is not much to see in the `DataFrameGroupBy` object that resulted. But there is a lot we can do with this object. Typing `grouped.` and hitting tab will show you the many possibilities. For most of these possibilities, the apply and combine steps happen together and a new `DataFrame` is returned. The `grouped.median()` method is exactly what we want.

In [12]:
df_median = grouped.median()

# Take a look
df_median

,participant number,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,rank grouped by insomnia
insomnia,,,,,,,,,,,,,,,
False,54,36,90,80,85.0,74.5,55.5,71.5,59.0,75.0,59.25,26,4,6,39.0
True,46,39,90,75,75.0,76.5,72.0,71.0,68.5,77.0,65.00,14,9,7,13.0


The outputted `DataFrame` has the medians of all quantities, including the `percent correct` that we wanted. Note that this DataFrame has `insomnia` as the name of the row index. If we want to instead keep `insomnia` (which, remember, is what we used to split up the data set before we computed the summary statistics) as a column, we can use the `reset_index()` method.

In [6]:
df_median.reset_index()

,insomnia,participant number,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess
0,False,54,36,90,80,85.0,74.5,55.5,71.5,59.0,75.0,59.25,26,4,6
1,True,46,39,90,75,75.0,76.5,72.0,71.0,68.5,77.0,65.00,14,9,7


We can also use multiple columns in our `groupby()` operation. For example, we may wish to look at four groups, male insomniacs, female insomniacs, male non-insomniacs, and female non-insomniacs. To do this, we simple pass in a list of columns into `df.groupby()`.

In [7]:
df.groupby(['gender', 'insomnia']).median().reset_index()

,gender,insomnia,participant number,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess
0,f,False,58.0,36.0,85.0,80.0,85.00,74.00,55.00,70.50,60.00,74.00,58.75,26.0,4.0,7.0
1,f,True,46.0,39.0,80.0,75.0,72.50,76.50,73.75,71.00,68.50,77.00,70.50,14.0,9.0,7.0
2,m,False,41.0,38.5,90.0,80.0,82.50,76.00,57.75,74.25,54.75,76.25,59.25,29.0,3.0,6.0
3,m,True,55.5,37.0,95.0,82.5,83.75,83.75,55.50,75.75,73.25,81.25,62.50,14.0,9.0,8.0


This type of operation is called an aggregation. That is, we split the data set into groups, and then compute a summary statistic for each group, in this case the median.

## Transformation
Instead of summarizing data in a group with single summary statistics by aggregation, we can also do a transformation in which each row gets a new entry within a given group. As a simple example, we could generate a column that gives the rank of each participant in terms of `percent correct` for an insomniac group versus non-insomniac groups. That is, we perform a rank ordering with the insomniac group and within the non-insomniac group.

In [8]:
grouped['percent correct'].rank()

0      11.0
1      21.5
2      23.0
3      19.5
4       3.5
5       2.0
6      17.0
7       6.0
8      21.5
9      12.5
10      3.5
11     24.0
12     15.0
13     15.0
14      5.0
15      8.5
16      8.5
17     15.0
18     18.0
19     12.5
20      8.5
21     25.0
22     19.5
23      1.0
24      8.5
25     15.0
26     41.0
27     19.0
28      9.5
29     72.0
       ... 
72     52.5
73     74.0
74     67.5
75     76.0
76      4.5
77     72.0
78     25.0
79     25.0
80     34.0
81      1.0
82     52.5
83     41.0
84     52.5
85     25.0
86      6.5
87     15.0
88     41.0
89     67.5
90     41.0
91     15.0
92     61.0
93     61.0
94      2.0
95     52.5
96      4.5
97     25.0
98     52.5
99     19.0
100    11.5
101     6.5
Name: percent correct, Length: 102, dtype: float64

This gave us a column of ranks with the indexing of the original `DataFrame` preserved. We can put this column into the `DataFrame`.

In [9]:
df['rank grouped by insomnia'] = grouped['percent correct'].rank(method='first')

# Take a look
df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence incorrect hit,confidence correct reject,confidence incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia,rank grouped by insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True,11.0
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True,21.0
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True,23.0
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True,19.0
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True,3.0


To verify that this worked correctly, and also to show some nice sorting properties of `DataFrames`, we will sort the `DataFrame` by `insomnia` and then by `percent correct` and make sure the ranks worked accordingly.

In [10]:
df_sorted = df.sort_values(by=['insomnia', 'percent correct'])

# Look at relevant columns
df_sorted[['insomnia', 'percent correct', 'rank grouped by insomnia']]

,insomnia,percent correct,rank grouped by insomnia
81,False,40.0,1.0
94,False,55.0,2.0
39,False,57.5,3.0
76,False,60.0,4.0
96,False,60.0,5.0
86,False,62.5,6.0
101,False,62.5,7.0
41,False,65.0,8.0
28,False,67.5,9.0
50,False,67.5,10.0
